#Preamble

This mini-project involves working through all the steps of a problem, whereas prior assignments asked you to just implement core functions. We will give you a dataset, but you will also have the opportunity to manipulate the data in ways that you find beneficial to the overall project and to explain why and how those manipulations mattered. This will be in addition to building the model from scratch, developing the training loop, and implementing testing. The code will be accompanied by a report written into the notebook.

This project will have you working with attention mechanism, in a new type of system for question-answering. This will provide you with experience working with attention mechanisms while not directly working with transformers.

This assignment is not autograded. You can modify any code cells as long as you achieve the requirements of each graded component.

# Introduction

Memory networks learn to access external memory stores (a database or, in the case of this assignment, a dictionary). Key-Value Memory Networks specifically assume that the external memory store is organized as a dictionary with keys and values. In theory memory networks are useful when one wants a neural network to be able to know a lot of information but we don't want to try to encode that information directly into the parameters of the network. This means information can be changed in the external memory database without retraining the neural network.

Given a question, e.g., "Where was Alexander Hamilton born?", a key-value memory network learns an embedding such that the question has a high cosine similarity to a particular key in the external dictionary. Because there are many keys that need to be matched against, key-value memory networks implement an attention-scoring mechanism to select a key. Because attention is a probabilistic score, the key-value memory network retrieves a sum of embeddings weighted according to the attention score. This weighted embedding is then compared to values using a second attention-scoring mechanism. The value with the highest cosine similarity can then be retrieved and returned as the answer.

Memory networks were an important part of the evolution of question-answering systems that have been eclipsed by transformers. However, the attention mechanism in a key-value memory network is very similar to the self-attention inside a transformer, so implementing a key-value memory network is a really great way to experiment and learn about self-attention without the added complexity of transformers.

Key-value memory networks are also closely related to retieval-based generation networks, except we will be retrieving facts from a dictionary instead of via the internet. However, the embedding of retrieved data will be similar.

Key-value memory networks are described in this [paper](https://arxiv.org/abs/1606.03126). It is recommended that you read the paper, but we will also walk through the steps you will need to complete.

# Some imports



You may add imports as necessary.

In [70]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import re
import os
import json
import random
import numpy as np
import pickle
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [71]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Unidecode is useful for getting rid of issues that arise from unicode. This should not be used if we care about unicode, but for the purposes of an instructional exercise, it eliminates a lot of edge cases that come up with unicode.

In [72]:
!pip install unidecode

In [73]:
import unidecode

If you need to have a reduced vocabulary, you can create an unknown "unk" token and add it to the vocabulary. Make sure the token index in the vocabulary and `UNK_ID` match.

In [74]:
UNK = 'unk'
UNK_ID = '0'

# Some utilities

You may edit these as necessary.

Stem words

In [75]:
# Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= [ps.stem(word) for word in text]
    return text

Simple tokenizer that only keeps letters and numbers.

In [76]:
def tokenize(line):
    line = re.sub(r'[^a-zA-Z0-9]', ' ', unidecode.unidecode(line)) # remove punctuation
    line = line.lower().split()  # lower case
    return line

A standard vocabulary object class

In [77]:
class Vocab:
    def __init__(self, name = 'vocab'):
        self.name = name
        self._word2index = {}
        self._word2count = {}
        self._index2word = {}
        self._n_words = 0

    def get_words(self):
      return list(self._word2count.keys())

    def num_words(self):
      return self._n_words

    def word2index(self, word):
      return self._word2index[word]

    def index2word(self, word):
      return self._index2word[word]

    def word2count(self, word):
      return self._word2count[word]

    def add_sentence(self, sentence):
        for word in tokenize(sentence):
            self.add_word(word)

    def add_word(self, word):
        if word not in self._word2index:
            self._word2index[word] = self._n_words
            self._word2count[word] = 1
            self._index2word[self._n_words] = word
            self._n_words += 1
        else:
            self._word2count[word] += 1

Make a bag of words frmo a sentence, given a vocabulary. Can return a bag of word counts or a a bag of word presences.

In [78]:
def multihot(s, vocab, preserve_counts = False):
  tokens = []
  for t in tokenize(s):
    if t in vocab._word2index:
      tokens.append(vocab.word2index(t))
    else:
      # Handle unknown words by skipping them or using UNK token if available
      if UNK in vocab._word2index:
        tokens.append(vocab.word2index(UNK))
      # Otherwise skip unknown words
  
  if not tokens:  # If no valid tokens found, return zero vector
    return np.zeros(vocab.num_words())
    
  tokens = np.array(tokens)
  mhot = np.zeros((tokens.size, vocab.num_words()))
  mhot[np.arange(tokens.size), tokens] = 1
  if preserve_counts:
    return mhot.sum(0)
  else:
    return mhot.sum(0) >= 1

If you have a reduced vocabulary, use this to replace out-of-vocab words. If you use this, you may want to merge it with `multihot` above to avoid tokenizing twice.

In [79]:
def unkit(s, vocab):
  return ' '.join(list(map(lambda x: UNK if x not in vocab._word2index else x, tokenize(s))))

# Part A: Download and Process Data (0 points)

This dataset contains the information in tables that are commonly used in Wikipedia biography pages. Each person has different rows of information pertaining to their notable accomplishments and details about their life. There are a large number of types of information that can appear as rows in the biography tables, however they are relatively uniform. We call the keys of the rows "relations".

For example [Alexander Hamilton](https://en.wikipedia.org/wiki/Alexander_Hamilton) has information about the President he worked for as Secretary of State, birth date, date of death, parents' names, etc.

The code below will download the dataset and process it to create two things:
- `DB`: a hash table that map titles of biography wikipedia articles to table information. The table information is represented as a nested hash table containing relations as keys, and associated values. For example, `DB['alexander hamilton'] = {'party': 'federalist',
 'spouse': 'elizabeth schuyler', ...}`
- `VOCAB`: A vocabulary object that maps words to tokens and vice versa.

In [80]:
# !git clone https://github.com/rlebret/wikipedia-biography-dataset.git

In [81]:
# !cat wikipedia-biography-dataset/wikipedia-biography-dataset.z?? > tmp.zip
# !unzip -o tmp.zip
# !rm tmp.zip

Get all the wikipedia titles.

In [82]:
train_titles = []
with open("wikipedia-biography-dataset/train/train.title", "r") as file:
  for line in file:
    train_titles.append(line.rstrip())

Boxes contain all the information, with each line corresponding to a title in `titles`.

In [83]:
train_boxes = []
with open("wikipedia-biography-dataset/train/train.box", "r") as file:
  for line in file:
    train_boxes.append(line.rstrip())

This will make the DB object, a dictionary of dictionaries for each wikipedia title, which is more or less the same as names. This function only keeps politicians (containing the "office" key term) and strips out information about images. It can be improved in many ways.

In [84]:
# Make a dictionary of dictionaries
def make_db(titles, boxes):
  db = {} # The DB
  # Iterate through titles
  for i in tqdm(range(len(titles))):
    box = boxes[i] # Grab the corresponding box information
    d  = {} # Inner dictionary
    # Build a dict for the ith entry
    # grab each key:value pair
    for pair in re.findall(r'([a-zA-Z_]+)[0-9]*\:([\w\d]+)', box):
      key, value = pair
      # Do a bit of cleaning
      key = key.strip()
      value = value.strip()
      # If the key contains the word image, we probably don't want to keep it
      if 'image' not in key:
        # The regex maintains underscores, strip those off
        if key[-1] == '_':
          key = key[:-1]
        # Make a new entry in inner dictionary if we don't have one
        if key not in d:
          d[key] = value
        # Keys with compound values are split up, which is annoying, so put them back together
        else:
          d[key] += ' ' + value
    # If it has an office key, keep it.
    if 'office' in d:
      db[titles[i]] = d
  return db

Build the vocab from the DB. Convert the whole thing into a string, tokenize it, and feed the surviving words into the vocab.

In [85]:
def make_vocab(DB):
  # Make the vocab object
  vocab = Vocab()
  # Tokenize the data by converting the entire DB into a string
  tokens = tokenize(str(DB))
  # Iterate through all the tokens (tqdm provides a progress bar)
  for t in tqdm(tokens):
    vocab.add_word(t)
  return vocab

If you want to discard rare words, this will rebuild the vocab. This is just an example.

In [86]:
def reduce_vocab(vocab, min_word_occurrence = 2):
  # make a new vocab
  vocab2 = Vocab("top")
  # Add the UNK token
  vocab2.add_word(UNK)
  # Iterate through vocabulary
  for w in list(vocab._word2count.keys()):
    count = vocab._word2count[w]
    idx = vocab._word2index[w]
    # If the word count passes threshold, add it to the new vocabulary object
    if count >= min_word_occurrence:
      vocab2.add_word(w)
      vocab2._word2count[w] = count
  # Return the new vocabulary object
  return vocab2

Make the DB

In [87]:
# DB = make_db(train_titles, train_boxes)

Make the VOCAB

In [88]:
# VOCAB = make_vocab(DB)
# # Add UNK token to vocabulary
# VOCAB.add_word(UNK)
# print(VOCAB.num_words())

## Save Processed Data

You may find it useful to save the processed dataset to your Google Drive.

It is recommended that you save the file to your Google Drive. To mount your Google Drive, open the file icon on the left side of the screen to get to the option). To save the file in your Google Drive use the path `'drive/MyDrive/filename'`.)

In [89]:
# from google.colab import drive
# drive.mount('/content/drive')

In [90]:
import os
# dump in containing directory
local_path = os.getcwd()

In [91]:
# with open(os.path.join(local_path, "data"), "wb") as f:
#   pickle.dump(DB, f, protocol=None, fix_imports=True, buffer_callback=None)

In [92]:
# with open(os.path.join(local_path, "vocab"), 'wb') as f:
#   pickle.dump(VOCAB, f, protocol=None, fix_imports=True, buffer_callback=None)

## Load processed data

If you have saved the processed data in your Google Drive, you can re-load it with these commands.

In [93]:
with open(os.path.join(local_path, "vocab"), "rb") as f:
  VOCAB = pickle.load(f)

In [94]:
with open(os.path.join(local_path, "data"), "rb") as f:
  DB = pickle.load(f)

## Data example

Get to know your data.

In [95]:
DB["alexander hamilton"]

{'name': 'alexander hamilton',
 'office': '1st united states secretary of the treasury senior officer of the army delegate to the congress of the confederation from new york',
 'president': 'george washington john adams',
 'term_start': 'september 11 1789 december 14 1799 november 3 1788 november 4 1782',
 'term_end': 'january 31 1795 june 15 1800 march 2 1789 june 21 1783',
 'predecessor': 'position established george washington egbert benson seat established',
 'successor': 'oliver wolcott jr james wilkinson seat abolished seat abolished',
 'birth_date': '11 january 1755',
 'birth_place': 'charlestown nevis british west indies',
 'death_date': 'july 12 1804 aged 47 or 49',
 'death_place': 'new york city new york u',
 'party': 'federalist',
 'spouse': 'elizabeth schuyler',
 'children': 'philip angelica alexander james alexander john church william stephen eliza holly phil',
 'alma_mater': 'kings college new york',
 'religion': 'presbyterian episcopalian convert',
 'signature': 'alexan

# Part B: Implement the Key-Value Memory Network (10 points)

This [paper](https://arxiv.org/abs/1606.03126) describes the key-value memory networks in detail, which is also sketched out below.

A key-value memory network takes a natural language question. This question will be converted into a bag-of-words (i.e., a multihot) Call this $x$ and it is a 1D tensor of vocabulary length.

![KVMemNet architecture](https://github.com/markriedl/kvmemnet-assignment/blob/32479dd1e88a9f8dfc72f11ccb8e9e0e1f78905f/kvmemnet-inside.png?raw=true)

The KVMemNet will contain a linear layer (or embedding layer) that will produce a 1D embedding of the question $q=A(x)$.

The KVMemNet will also take in a stack of keys as a tensor of shape `num_keys x vocab_size`. Each row is embedded using the same embedding, $k=A(keys)$, producing a tensor of shape `num_keys x embed_dim`. How this stack of keys is chosen will be discussed below.

The KVMemNet will take in a third input, a stack of values associated with the stack of keys. This will also be of shape `num_values x vocab_size`. Each row is embedded using the same embedding, $v=A(values)$, producing a tensor of shape `num_keys x embed_dim`.

The KVMemNet will also contain a second linear embedding layer, $B$. More on this later.

Once we have `q`, `k`, and `v` embeddings, the next step is to use `q` and `k` to compute attention scores that can be applied against `v`. Think of $A$ as learning how to make questions and the keys that should match against values that have received the same treatment.

The attention scores `p` are computed by taking the inner-product (`torch.inner()`) between `q` and `k`. The result will be a 1D tensor with `num_keys` length. Use softmax so that `p` contains scores between 1.0 and 0.0.

You may be wondering why there isn't a non-linearity such as a sigmoid or ReLU after the linear layer. Softmax is a non-linearity.

Next apply the `p` attention scores against `v` to apply a weight against each value in the stack of values. One should be highly weighted and the rest less weighted. Sum all the weighted values up to create a 1D tensor `o` of feature weights of length `embed_dim`. `p` can be thought of as how much of each value gets selected. Then they all get combined together and the feature weights are proportional to how much each value was attended to. The `torch.matmul()` can do the multiplication and summing in one step.

The KVMemNet forward function should return this tensor of feature weights `o`.

A quick note on `k` and `v`. We can't send the entire set of keys and values in our database through the network's forward function. Instead there should be a selection mechanism that selects just a subset of the database. This subset should contain the best key for the question $x$ to match against, and its corresponding value. We assume that a shallow selection process can narrow down the key-value pairs to a relatively small set, one of which will be best. For example, if the question involves "Alexander Hamilton", we can reasonably guess that the best key-value pair is in the part of the database associated with the named person.

We are not done though. What about our linear layer $B$? Suppose variable `Y` contains our entire set of values in our databse as bags of words. $B$ is going to be used to embed our entire set of database values $y=B(Y)$. $B$ can be thought of as learning how to make all the values look like the feature weights output by the model such that the highest cosine similarity corresponds to the correct value taken from *all* values in the database.

$B$ should live inside the KVMemNet object so that its parameters become trainable, but notice that we do not use $B$ in the KVMemNet's forward function. $B$ will get used to prepare the stack of all values in the database for training. It will bet used in the training loop but outside of the forward function. This is a bit unusual, but necessary to figure out the correct target (the true index of the best value to match against) for training.

The above explantion only implements *single-hop* retrieval. *multi-hop* retrieval allows the results of one retrieval to inform a second (and third and so on) to get the right retrieval. This would be used in the case where the answer cannot be inferred directly from the question in a single retrieval, such as "What was the founding date of the country that Alexander Hamilton was born in?". To implement multi-hop retrieval, the KVMemNet will have additional linear layers $R_1...R_n$. Each $R_{i}$ will do a linear transform on `q` then attention will score and retrieve values as feature weights `o`. This will be sent to the next $R_{i+1}$ and so on until the hops are complete. This final `o` will be returned.

For this assignment is is sufficient to only do *single-hop* retrieval.

The above explanation does not include consideration of batching. You may want to add a batch dimension as the first dimension and input a batch as a set of questions, a set of stacks of keys, and a set of stacks of values. To do this, functions like `.inner()`, `.mm()`, and`.matmul()` will not work. Instead use `.bmm()` which handles batching correctly. You will probably need to do some `.squeeze()` and `.unsqueeze()` operations to make sure your tensors are the correct shapes.

Instead of bag-of-words, one may also consider first converting each question, key, and value into a general set of embeddings such as [GLoVe](https://nlp.stanford.edu/projects/glove/). To do this one will need to consider how to combine words--convert each word into an embedding vector and then add the vectors together (or maybe average them).

**Complete the key-value memory net code**

In [ ]:
class KVMemNet(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(KVMemNet, self).__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        ### YOUR CODE HERE
        self.A = nn.Linear(vocab_size, embed_dim)
        self.B = nn.Linear(vocab_size, embed_dim)

    def forward(self, x, keys, values):
        # pseudocode from OH:
        # questions = A(x)
        # keys = A(keys) 
        # values = A(values)
        
        # prob = questions * keys
        # prob = softmax(prob)
        # output = prob * values
        
        # Handle both single samples and batches
        if x.dim() == 1:
            x = x.unsqueeze(0)
            keys = keys.unsqueeze(0) 
            values = values.unsqueeze(0)
            single_sample = True
        else:
            single_sample = False

        # Embed using layer A
        questions = self.A(x)  # [batch, embed_dim]
        keys_embed = self.A(keys)  # [batch, num_keys, embed_dim]
        values_embed = self.A(values)  # [batch, num_keys, embed_dim]
        
        # Compute attention scores
        # [batch, num_keys, embed_dim] x [batch, embed_dim, 1] = [batch, num_keys, 1]
        prob = torch.bmm(keys_embed, questions.unsqueeze(2))  
        prob = prob.squeeze(2)  # [batch, num_keys]
        
        # Apply softmax
        prob = F.softmax(prob, dim=1)  # [batch, num_keys]
        
        # Weight values by attention and sum
        # [batch, 1, num_keys] x [batch, num_keys, embed_dim] = [batch, 1, embed_dim]
        output = torch.bmm(prob.unsqueeze(1), values_embed)
        output = output.squeeze(1)  # [batch, embed_dim]
        
        if single_sample:
            output = output.squeeze(0)
            
        return output


# Synthetic Data Set

This is a synthetic dataset. One way to test a model during development is to take a small piece of data and show that you can overfit a model. If you can't overfit an easily learned chunk of data, then you probably have something wrong in your code. In this case I have provided a small chunk of synthetic data that should be easy to learn.

- The vocabulary is 20 word: 5 names, 5 relations, 5 question-words, 5 values
 - First 5 elements of the vocab are names (for example index 0 might be "Hamilton").
 - Second 5 elements of the vocab are relations (for example, "born", "died", "occupation").
 - Third 5 elements are random words that might be part of a query (for example, "When was").
 - Final 5 elements of the vocab are possible values (for example, "1757")
- A "question" is a name (5, 1), relation (5, 1), some words (5, 1), and no values
- The keys will all have the same name (5, 5) where each row is idential, relations (5, 5), no words, no values
- Values will have no names, no relations, no words, and value vocab words

In [97]:
# Turn on a different relation on each row
relations = torch.zeros(5, 5)
relations.fill_diagonal_(1)

# training data
train_data = {}
for i in range(5):
  # Name associated with questions, keys, values
  train_data[i] = (torch.cat([F.one_hot(torch.arange(0, 5))[i].repeat(5, 1),
                         relations,
                         torch.randint(0, 2, (5, 5)).float(),
                         torch.zeros(5, 5)], dim=1),
              torch.cat([F.one_hot(torch.arange(0, 5))[i].repeat(5, 1),
                         relations,
                         torch.zeros(5, 5),
                         torch.zeros(5, 5)], dim=1),
              torch.cat([torch.zeros(5, 5),
                         torch.zeros(5, 5),
                         torch.zeros(5, 5),
                         torch.randint(0, 2, (5, 5)).float()], dim=1))
  Y = torch.cat([v[2] for v in list(train_data.values())], dim=0).to('cuda')

# Part C: Train on Synthetic Data (5 points)

The following describes the steps to set up a training loop, including the training of the $B$ layer.

![The KVMemNet being used in the training loop](https://github.com/markriedl/kvmemnet-assignment/blob/main/kvmemnet-outside.png?raw=true)

- Create a model with the given vocabulary size and an embedding size that is equal to or smaller.
- Loop through `N` epochs:
 - There are five names, loop through each name.
   - Get a stack of questions, stack of keys, and stack of values from `DB_synth`.
   - Loop through the relations. There is relation on each row of the keys and values.
     - Get a single question, the `i`-th row in the questions pulled from `DB_synth` above.
     - Compute the target: this is the `name*5 + i` element in `Y`.
     - Run the singular question, stack of keys, and stack of values through the model and produce an output, which is a tensor of feature weights.
     - Run all of `Y` through `model.B()` to get an embedded stack of values.
     - Take the softmax of the inner product between the embedded stack of values from `Y` and the feature weight generated by the model.
     - Compute the loss with `nn.CrossEntropyLoss`.
     - Call `.backward()` on the loss.

In addition to printing the loss (after every question or after every name in `DB_synth`) you can also print the target and the argmax of the softmax result to see if they match. Over time you should see the target and the argmax in agreement. For the purposes of this part of the project it is sufficient to test on the training set.

Don't forget to move the model and the tensor to the GPU.

You may want to speed up training by implementing batching. To do this, the model `forward()` needs to take tensors with an extra batching dimension as the first dimension. However, `.inner()`, `.mm()`, and `.matmul()` will not work properly. You will need to use `.bmm()` instead, which understands the first dimension is for batching. You will likely find that you need to perform some `.squeeze()` and `.unsqueeze()` operations. You can try batch-size of one, or take entire chunks (or even all synthetic data as a single, large batch). Try it different ways.

Try training on the synthetic data first.

You may make as many cells as necessary. Save your notebook outputs that plot loss and show it reducing.

**Write code blocks below that create the `KVMemNet`**

In [ ]:
# Pseudo from OH
"""
vocab_size = 20
embed_size = 16
model = KVMemNet(vocab_size, embed_size)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
"""

vocab_size = 20
embed_dim = 16
learning_rate = 0.001
num_epochs = 100  # Increased for better convergence

model = KVMemNet(vocab_size, embed_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

**Write and run a training testing loop. Show that your training loop loss converges with a plot**

To plot a loss curve, compute the mean loss per epoch and save it in a list:
```
x_axis.append(epoch_number)
y_axis.append(mean_epoch_loss_for_this_epoch)
plt.plot(x_axis, y_axis)
plt.show()
```

In [ ]:
# Write your training loop here
Y_tensor = Y.to(device)

# Initialize loss tracking lists for plotting
losses = []  # Loss per batch/sample
epoch_losses = []  # Average loss per epoch

# Pseudo from OH
"""
for epoch in range(500):
    for name_index in range(5):
        qs, keys, values = train_data[name_index]
        keys = keys.to(device)
        values = values.to(device)
        
        for relation_index in range(5):
            q = qs[relation_index].to(device)
            output = model(q, keys, values)
            y = model.B(Y)
            
            soft = softmax(inner(y, output))
            target_index = name_index * 5 + relation_index
            
            soft.unsqueeze if necessary
            loss = criterion(soft, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            print('prediction', soft.argmax(), 'target', target_index, 'loss', loss.item())
"""

# Training loop based on OH pseudocode
for epoch in range(num_epochs):
    epoch_loss = 0.0
    correct = 0
    total = 0
    
    # Loop through 5 names
    for name_index in range(5):
        # Get questions, keys, values for this person
        qs, keys, values = train_data[name_index]
        
        # Move to device
        qs = qs.to(device)
        keys = keys.to(device)
        values = values.to(device)
        
        # Process each relation/question for this person
        for relation_index in range(5):
            # Get single question
            q = qs[relation_index]  # Shape: [20]
            
            # Forward pass through model
            output = model(q, keys, values)  # Shape: [embed_dim]
            
            # Embed all Y values using layer B
            y = model.B(Y_tensor)  # Shape: [25, embed_dim]
            
            # Compute inner product and softmax
            scores = torch.inner(y, output)  # Shape: [25]
            
            # Target index for this name-relation combination
            target_index = name_index * 5 + relation_index
            target = torch.tensor([target_index], dtype=torch.long, device=device)
            
            # Add batch dimension for CrossEntropyLoss
            scores = scores.unsqueeze(0)  # Shape: [1, 25]
            
            # Compute loss
            loss = criterion(scores, target)
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Track metrics
            prediction = scores.argmax(dim=1).item()
            correct += (prediction == target_index)
            total += 1
            
            loss_value = loss.item()
            losses.append(loss_value)
            epoch_loss += loss_value
            
            # Print progress occasionally
            if epoch % 20 == 0 and name_index == 0 and relation_index == 0:
                print(f'Epoch {epoch}, prediction: {prediction}, target: {target_index}, loss: {loss_value:.4f}')
    
    # Track epoch average
    avg_loss = epoch_loss / 25
    epoch_losses.append(avg_loss)
    
    # Print epoch summary
    if epoch % 20 == 0:
        accuracy = 100.0 * correct / total
        print(f"Epoch [{epoch}/{num_epochs}] - Avg Loss: {avg_loss:.4f}, Accuracy: {accuracy:.1f}%")
        print("-" * 50)

# Plot results
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(losses[-500:])  # Show last 500 samples
plt.title('Training Loss (Last 500 Samples)')
plt.xlabel('Sample')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(epoch_losses)
plt.title('Average Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')

plt.tight_layout()
plt.show()

print(f"Final average loss: {epoch_losses[-1]:.4f}")
print(f"Final accuracy: {100.0 * correct / total:.1f}%")

# Part D: Training on the Full Data (10 points)

To train on the full data, you are going to need to do some pre-processing of the data.

First, there are no "questions". You need to generate questions for each type of relation. There a number of ways to do this. The simplest is to just assume that a question is the name of a person and a relation, e.g., "Alexander Hamilton birth date". Another way would be to create templates for each type of relation. For example the "birth date" relation would have the following template: "When was [name] born?", filling in the [name]. Because there are a lot of different types of relations, you may want to remove the more obscure relations so you need fewer templates and also have a smaller vocabulary. Templates work well if the questions are expected to be almost identical to the templates. You may want to generate multiple templates per relation. Continuing the previous example, a second template would be: "What is the birthdate of [name]?".

If you are feeling more ambitious, you could use GPT-J, GPT-NeoX, GPT-3 or ChatGPT to generate templates. It works decently well and you can get some variety of templates.

The question should contain information about the person and some words that are representative of the relation even if the exact relation words aren't used (the KVMemNet should figure out that "birthdate" and "born" are correlated).

You only put a subset of all key-value pairs into the KVMemNet. You need a technique for sub-selecting from all the key-value pairs in `DB`. You might just need the ones that are directly associated with the person (Alexander Hamilton has 23). You may need to mix in a few key-value pairs from another person's entries in the database to help ensure against accidental overfitting.

The final challenge you will have in the training loop is that there may still be too many unique values in `Y` to encode and create one big tensor. In that case, you can at least use the values that you sent to the KVMemNet, along with as many other randomly selected values as you can fit into the GPU's memory.

Create as many cells below as you need. Save the output of your training and testing functions, reporting loss during training and accuracy during testing. 5 points for a training loop that reduces loss. 5 points for a training function with a correct accuracy computation.

**Create a training dataset and a non-overlapping testing dataset**

If CPU memory becomes a problem you might want to consider a `DataLoader` so that data can be stored on file and pulled up when needed.

In [100]:
# Create your training and test sets here
import torch.utils.data as data

# Simplified question templates to avoid too many OOV words
relation_templates = {
    'birth': ['{name} birth', 'when {name} born', '{name} birth date'],
    'birthplace': ['{name} birthplace', 'where {name} born', '{name} birth place'],
    'death': ['{name} death', 'when {name} died', '{name} death date'],
    'deathplace': ['{name} deathplace', 'where {name} died', '{name} death place'],
    'office': ['{name} office', '{name} position', '{name} job'],
    'party': ['{name} party', '{name} political party'],
    'spouse': ['{name} spouse', '{name} married', '{name} wife'],
    'successor': ['{name} successor', 'who succeeded {name}'],
    'predecessor': ['{name} predecessor', 'who preceded {name}'],
    'education': ['{name} education', '{name} school', '{name} studied'],
    'occupation': ['{name} occupation', '{name} job', '{name} work'],
    'religion': ['{name} religion', '{name} religious'],
    'nationality': ['{name} nationality', '{name} country'],
    'residence': ['{name} residence', '{name} lived', '{name} home'],
    'alma_mater': ['{name} alma mater', '{name} graduated']
}

def create_question_from_template(name, relation):
    """Create a question using templates for a given name and relation"""
    if relation in relation_templates:
        template = random.choice(relation_templates[relation])
        return template.format(name=name)
    else:
        # Default template for unknown relations
        return f"{name} {relation}"

def multihot_torch_optimized(s, vocab):
    """
    Optimized PyTorch-based multihot encoding that's much faster than NumPy version
    Creates tensors directly without NumPy intermediate steps
    """
    tokens = tokenize(s)
    
    # Build list of valid indices
    indices = []
    for t in tokens:
        if t in vocab._word2index:
            indices.append(vocab.word2index(t))
        elif UNK in vocab._word2index:
            indices.append(vocab.word2index(UNK))
    
    if not indices:
        return torch.zeros(vocab.num_words(), dtype=torch.float32)
    
    # Create tensor directly with PyTorch operations
    mhot = torch.zeros(vocab.num_words(), dtype=torch.float32)
    indices_tensor = torch.tensor(indices, dtype=torch.long)
    mhot.index_fill_(0, indices_tensor, 1.0)
    
    return mhot

class KVMemNetDatasetPersonBased(data.Dataset):
    """
    Dataset that properly groups data by person, then samples relations
    Each sample is still a (question, keys, values) tuple for a specific relation
    """
    
    def __init__(self, names, DB, vocab, relation_templates, max_key_pairs=10):
        self.DB = DB
        self.vocab = vocab
        self.relation_templates = relation_templates
        self.max_key_pairs = max_key_pairs
        
        # Build person-based structure
        self.persons = []  # List of (name, valid_relations) tuples
        self.samples = []  # Flattened list of (person_idx, relation) for training
        
        # Process each person
        for person_idx, name in enumerate(names):
            if name in DB:
                person_data = DB[name]
                valid_relations = [r for r in person_data.keys() if r in relation_templates]
                
                if valid_relations:
                    self.persons.append((name, valid_relations))
                    # Create samples for each relation of this person
                    for relation in valid_relations:
                        self.samples.append((len(self.persons) - 1, relation))
        
        print(f"Created dataset with {len(self.persons)} persons and {len(self.samples)} total samples")
        
        # Pre-compute list of person indices for random sampling
        self.person_indices = list(range(len(self.persons)))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        """
        Get a sample for training. Each sample is one question about one relation
        for a specific person, but includes all their relations as keys/values
        """
        person_idx, target_relation = self.samples[idx]
        person_name, person_relations = self.persons[person_idx]
        person_data = self.DB[person_name]
        
        # Create question for the target relation
        question_text = create_question_from_template(person_name, target_relation)
        question_tensor = multihot_torch_optimized(question_text, self.vocab)
        
        # Prepare keys and values
        keys = []
        values = []
        
        # Add ALL relations for this person (target first)
        # First add the target relation
        target_key = f"{person_name} {target_relation}"
        target_value = person_data[target_relation]
        keys.append(multihot_torch_optimized(target_key, self.vocab))
        values.append(multihot_torch_optimized(target_value, self.vocab))
        
        # Then add other relations from the same person
        for rel in person_relations:
            if rel != target_relation:
                key_text = f"{person_name} {rel}"
                value_text = person_data[rel]
                keys.append(multihot_torch_optimized(key_text, self.vocab))
                values.append(multihot_torch_optimized(value_text, self.vocab))
        
        # If we have fewer than max_key_pairs, add some from other people
        if len(keys) < self.max_key_pairs:
            # Use deterministic random sampling based on idx
            rng = np.random.RandomState(idx)
            other_person_indices = [i for i in self.person_indices if i != person_idx]
            
            if other_person_indices:
                # Shuffle and select other persons
                rng.shuffle(other_person_indices)
                
                for other_idx in other_person_indices:
                    if len(keys) >= self.max_key_pairs:
                        break
                    
                    other_name, other_relations = self.persons[other_idx]
                    if other_relations:
                        # Pick a random relation from this other person
                        random_rel = rng.choice(other_relations)
                        random_key = f"{other_name} {random_rel}"
                        random_value = self.DB[other_name][random_rel]
                        
                        keys.append(multihot_torch_optimized(random_key, self.vocab))
                        values.append(multihot_torch_optimized(random_value, self.vocab))
        
        # Ensure we have exactly max_key_pairs (truncate if necessary)
        keys = keys[:self.max_key_pairs]
        values = values[:self.max_key_pairs]
        
        # Pad with zeros if still not enough
        while len(keys) < self.max_key_pairs:
            keys.append(torch.zeros(self.vocab.num_words(), dtype=torch.float32))
            values.append(torch.zeros(self.vocab.num_words(), dtype=torch.float32))
        
        # Stack tensors
        keys_tensor = torch.stack(keys)
        values_tensor = torch.stack(values)
        
        return {
            'question': question_tensor,
            'keys': keys_tensor,
            'values': values_tensor,
            'answer_text': target_value,
        }

def create_value_matrix_efficient(DB, vocab, max_values=5000):
    """Create a matrix of unique values more efficiently"""
    value_list = []
    value_to_idx = {}
    
    # Collect unique values
    for name, person_data in DB.items():
        for relation, value in person_data.items():
            if relation in relation_templates and len(value_list) < max_values:
                if value not in value_to_idx:  # Check using dict for O(1) lookup
                    value_to_idx[value] = len(value_list)
                    value_list.append(value)
    
    # Convert to tensor matrix using optimized function
    print(f"Creating Y matrix with {len(value_list)} unique values...")
    Y_list = []
    
    # Process in batches to show progress
    batch_size = 500
    for i in tqdm(range(0, len(value_list), batch_size), desc="Building Y matrix"):
        batch_end = min(i + batch_size, len(value_list))
        for j in range(i, batch_end):
            Y_list.append(multihot_torch_optimized(value_list[j], vocab))
    
    Y_matrix = torch.stack(Y_list)
    print(f"Y matrix created with shape: {Y_matrix.shape}")
    
    return Y_matrix, value_list, value_to_idx

# Create datasets
print("Creating training and test datasets...")

# Split names into train/test
all_names = list(DB.keys())
random.seed(42)  # For reproducibility
random.shuffle(all_names)
split_idx = int(len(all_names) * 0.8)
train_names = all_names[:split_idx]
test_names = all_names[split_idx:]

print(f"Train names: {len(train_names)}, Test names: {len(test_names)}")

# Create person-based datasets
train_dataset = KVMemNetDatasetPersonBased(train_names, DB, VOCAB, relation_templates)
test_dataset = KVMemNetDatasetPersonBased(test_names, DB, VOCAB, relation_templates)

# Create Y matrix efficiently
Y_matrix, Y_values, value_to_idx = create_value_matrix_efficient(DB, VOCAB, max_values=5000)

print(f"Dataset creation completed!")
print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Y matrix shape: {Y_matrix.shape}")

# Create DataLoaders without multiprocessing to avoid memory issues
# Set num_workers=0 to prevent memory duplication
train_loader = data.DataLoader(
    train_dataset, 
    batch_size=32, 
    shuffle=True, 
    num_workers=0,  # Critical: no multiprocessing to avoid memory leak
    pin_memory=True
)

test_loader = data.DataLoader(
    test_dataset, 
    batch_size=64, 
    shuffle=False, 
    num_workers=0,  # Critical: no multiprocessing to avoid memory leak
    pin_memory=True
)

Creating training and test datasets...
Train names: 32628, Test names: 8158
Created dataset with 32628 persons and 154746 total samples
Created dataset with 8158 persons and 38783 total samples
Creating Y matrix with 5000 unique values...


Building Y matrix: 100%|██████████| 10/10 [00:00<00:00, 24.19it/s]


Y matrix created with shape: torch.Size([5000, 96094])
Dataset creation completed!
Training samples: 154746
Test samples: 38783
Y matrix shape: torch.Size([5000, 96094])


**Create your `KVMemNet`**

In [101]:
# Set up your KVMemNet here
vocab_size = VOCAB.num_words()
embed_dim = 128  # Larger embedding for real data
learning_rate = 0.001

# Create model and move to GPU
real_model = KVMemNet(vocab_size, embed_dim).to(device)

# Setup optimizer and loss criterion
real_optimizer = optim.Adam(real_model.parameters(), lr=learning_rate)
real_criterion = nn.CrossEntropyLoss()

print(f"Real data model created with vocab_size={vocab_size}, embed_dim={embed_dim}")
print(f"Model moved to device: {device}")
print(f"Optimizer: Adam with lr={learning_rate}")
print(f"Loss function: CrossEntropyLoss")

Real data model created with vocab_size=96094, embed_dim=128
Model moved to device: cuda
Optimizer: Adam with lr=0.001
Loss function: CrossEntropyLoss


**Write and run a training loop, showing a loss plot**

You may find it handy to also test your network on the test data periodically as it trains.

In [102]:
def save_model(model, filename):
    """Save the model state dictionary to a file."""
    torch.save(model.state_dict(), filename)
    print(f"Model saved to {filename}")

def save_checkpoint(model, optimizer, epoch, filename):
    """Save the model and optimizer state to a checkpoint file."""
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch
    }
    torch.save(checkpoint, filename)
    print(f"Checkpoint saved to {filename}")

In [103]:
# Simplified training loop for real data following OH pattern
def train_model_simple(model, train_loader, Y_matrix, value_to_idx, 
                      optimizer, criterion, num_epochs=5, device='cuda'):
    """
    Simplified training loop that follows the OH pseudocode pattern
    """
    model.train()
    
    losses = []
    epoch_losses = []
    
    # Move Y_matrix to device once
    Y_matrix = Y_matrix.to(device)
    
    print(f"Starting training for {num_epochs} epochs...")
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        correct = 0
        total = 0
        
        # Progress bar for batches
        progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, batch in enumerate(progress):
            # Get batch data
            questions = batch['question'].to(device)
            keys = batch['keys'].to(device)
            values = batch['values'].to(device)
            answer_texts = batch['answer_text']
            
            # Find target indices
            targets = []
            valid_mask = []
            
            for i, answer_text in enumerate(answer_texts):
                if answer_text in value_to_idx:
                    targets.append(value_to_idx[answer_text])
                    valid_mask.append(i)
            
            if not targets:
                continue
                
            # Filter to valid samples
            targets = torch.tensor(targets, dtype=torch.long, device=device)
            questions = questions[valid_mask]
            keys = keys[valid_mask]
            values = values[valid_mask]
            
            # Forward pass (batched)
            output = model(questions, keys, values)  # [batch, embed_dim]
            
            # Embed Y values
            y_embedded = model.B(Y_matrix)  # [num_values, embed_dim]
            
            # Compute scores for all samples in batch
            # [batch, embed_dim] x [num_values, embed_dim]^T = [batch, num_values]
            scores = torch.matmul(output, y_embedded.T)
            
            # Compute loss
            loss = criterion(scores, targets)
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Track metrics
            predictions = scores.argmax(dim=1)
            correct += (predictions == targets).sum().item()
            total += len(targets)
            
            batch_loss = loss.item()
            losses.append(batch_loss)
            epoch_loss += batch_loss * len(targets)
            
            # Update progress bar
            accuracy = 100.0 * correct / total if total > 0 else 0
            progress.set_postfix({'Loss': f'{batch_loss:.4f}', 'Acc': f'{accuracy:.2f}%'})
        
        # Epoch statistics
        avg_loss = epoch_loss / total if total > 0 else 0
        epoch_accuracy = 100.0 * correct / total if total > 0 else 0
        epoch_losses.append(avg_loss)
        
        print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, Accuracy = {epoch_accuracy:.2f}%")
    
    return losses, epoch_losses

# Create a simpler model configuration
vocab_size = VOCAB.num_words()
embed_dim = 128  # Reasonable size

# Initialize model
real_model = KVMemNet(vocab_size, embed_dim).to(device)

# Simple optimizer
real_optimizer = optim.Adam(real_model.parameters(), lr=0.001)
real_criterion = nn.CrossEntropyLoss()

print(f"Model initialized with vocab_size={vocab_size}, embed_dim={embed_dim}")

# Train the model
train_losses, train_epoch_losses = train_model_simple(
    real_model, train_loader, Y_matrix, value_to_idx,
    real_optimizer, real_criterion, num_epochs=3, device=device
)

# Plot training curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
# Show a subset of losses for clarity
if len(train_losses) > 1000:
    plt.plot(train_losses[::len(train_losses)//1000])
else:
    plt.plot(train_losses)
plt.title('Training Loss Samples')
plt.xlabel('Sample')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(train_epoch_losses, 'bo-')
plt.title('Average Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Training completed! Final average loss: {train_epoch_losses[-1]:.4f}")

Model initialized with vocab_size=96094, embed_dim=128
Starting training for 3 epochs...


Epoch 1/3:  20%|██        | 981/4836 [04:01<15:47,  4.07it/s, Loss=6.3018, Acc=4.90%]


KeyboardInterrupt: 

**Write the code for testing your model on the test data**

Your training loop can call the testing loop. But make sure that you do one last test on the model after training completes.

In [ ]:
# Simplified testing loop
def evaluate_model_simple(model, test_loader, Y_matrix, value_to_idx, 
                         criterion, device='cuda'):
    """
    Simple evaluation following the same pattern as training
    """
    model.eval()
    
    correct = 0
    total = 0
    total_loss = 0
    
    # Move Y_matrix to device
    Y_matrix = Y_matrix.to(device)
    
    print("Evaluating model on test set...")
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            # Get batch data
            questions = batch['question'].to(device)
            keys = batch['keys'].to(device)
            values = batch['values'].to(device)
            answer_texts = batch['answer_text']
            
            # Find target indices
            targets = []
            valid_mask = []
            
            for i, answer_text in enumerate(answer_texts):
                if answer_text in value_to_idx:
                    targets.append(value_to_idx[answer_text])
                    valid_mask.append(i)
            
            if not targets:
                continue
                
            # Filter to valid samples
            targets = torch.tensor(targets, dtype=torch.long, device=device)
            questions = questions[valid_mask]
            keys = keys[valid_mask]
            values = values[valid_mask]
            
            # Forward pass
            output = model(questions, keys, values)
            
            # Embed Y values
            y_embedded = model.B(Y_matrix)
            
            # Compute scores
            scores = torch.matmul(output, y_embedded.T)
            
            # Compute loss
            loss = criterion(scores, targets)
            total_loss += loss.item() * len(targets)
            
            # Compute accuracy
            predictions = scores.argmax(dim=1)
            correct += (predictions == targets).sum().item()
            total += len(targets)
    
    accuracy = correct / total if total > 0 else 0
    avg_loss = total_loss / total if total > 0 else 0
    
    print(f"\nTest Results:")
    print(f"Accuracy: {accuracy:.4f} ({correct}/{total})")
    print(f"Average Loss: {avg_loss:.4f}")
    
    return accuracy, avg_loss

# Evaluate the model
test_accuracy, test_loss = evaluate_model_simple(
    real_model, test_loader, Y_matrix, value_to_idx, 
    real_criterion, device=device
)

# Show some example predictions
def show_examples(model, test_dataset, Y_matrix, Y_values, num_examples=5):
    """Show a few example predictions"""
    model.eval()
    
    print("\nExample predictions:")
    print("=" * 60)
    
    indices = np.random.choice(len(test_dataset), num_examples, replace=False)
    
    with torch.no_grad():
        for idx in indices:
            sample = test_dataset[idx]
            
            # Get data
            question = sample['question'].unsqueeze(0).to(device)
            keys = sample['keys'].unsqueeze(0).to(device)
            values = sample['values'].unsqueeze(0).to(device)
            answer_text = sample['answer_text']
            
            # Forward pass
            output = model(question, keys, values)
            
            # Get scores
            Y_matrix_gpu = Y_matrix.to(device)
            y_embedded = model.B(Y_matrix_gpu)
            scores = torch.matmul(y_embedded, output.squeeze())
            
            predicted_idx = torch.argmax(scores).item()
            
            if predicted_idx < len(Y_values):
                predicted_text = Y_values[predicted_idx]
            else:
                predicted_text = "Unknown"
            
            print(f"Actual: {answer_text}")
            print(f"Predicted: {predicted_text}")
            print(f"Match: {'✓' if answer_text == predicted_text else '✗'}")
            print("-" * 60)

show_examples(real_model, test_dataset, Y_matrix, Y_values, num_examples=5)

print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")
print(f"Final Test Loss: {test_loss:.4f}")

**Suggestion:** Once you have a model that has decent accuracy, you may want to save it to your Google Drive using ``torch.save()`` and load it when working on the next part of the assignment using ``torch.load()``.

# Part E: Use the Model (5 points)

Given a question in natural language, turn it into a bag of words and feed it into the model with a set of plausible keys and values. Apply the output feature embedding to the full set of values and pick the value with argmax. Return the actual text inside that value (not the bag of words or embedding).

That is, given a natural language question, you are asked to create the $q$ and pick a relevant subset of $k$ and $v$. Run the $q$, $k$, and $v$ through the model and get an answer to the original question.

For example a question might be "When was Alexander Hamilton born?" Depending on how you pre-proessed your data, you may need to extract the entity and the relation.

Write a function that takes in the `question` below, the data, and the model, and outputs the text answer, e.g., "11 january 1755". You must use your ``KVMemNet``.

**Suggestion:** To process a question you will probably want to find the entity and the relation. You may use packages such as [NLTK](https://www.nltk.org/) (already imported), [SpaCY](https://spacy.io/), [Stanza](https://stanfordnlp.github.io/stanza/), or other.

Change the question to test your implementation, but don't delete this cell.

In [ ]:
question = "When was alexander hamilton born?"

**Create your function for using the `KVMemNet` to answer a given question.**

The function should take in the question, data, model, and any other parameters you need. The function should return a text string.

You can create as many cells as necessary. Save the notebook cells showing one example of your input question and output answer. For grading we will look to see that your question is in natural language, the model is used, and the answer is in text. The example doesn't have to be correct. You will analyze your technique later in the report.

In [ ]:
# Create your functions here

def extract_entity_and_relation(question):
    """
    Extract the entity (person name) and relation type from a natural language question
    """
    question_lower = question.lower()
    
    # Common question patterns and their corresponding relations
    relation_patterns = {
        'birth': ['born', 'birth', 'birthdate', 'birth date'],
        'birthplace': ['birthplace', 'birth place', 'where.*born'],
        'death': ['died', 'death', 'deathdate', 'death date'],
        'deathplace': ['deathplace', 'death place', 'where.*died'],
        'office': ['office', 'position', 'job', 'role'],
        'party': ['party', 'political party'],
        'spouse': ['married', 'spouse', 'wife', 'husband'],
        'successor': ['successor', 'succeeded'],
        'predecessor': ['predecessor', 'preceded'],
        'education': ['education', 'school', 'university', 'studied'],
        'occupation': ['occupation', 'profession', 'job', 'work'],
        'religion': ['religion', 'religious'],
        'nationality': ['nationality', 'country', 'from'],
        'residence': ['residence', 'lived', 'home'],
        'alma_mater': ['alma mater', 'graduated', 'degree']
    }
    
    # Try to find relation based on keywords
    detected_relation = None
    for relation, patterns in relation_patterns.items():
        for pattern in patterns:
            if pattern in question_lower:
                detected_relation = relation
                break
        if detected_relation:
            break
    
    # Extract entity - look for capitalized words that might be names
    tokens = question.split()
    entity_candidates = []
    
    for i, token in enumerate(tokens):
        # Look for capitalized words that are not at the beginning of sentences
        if token[0].isupper() and token.lower() not in ['when', 'where', 'what', 'who', 'how', 'why']:
            # Check if next token is also capitalized (likely part of name)
            if i + 1 < len(tokens) and tokens[i + 1][0].isupper():
                entity_candidates.append(f"{token} {tokens[i + 1]}")
            else:
                entity_candidates.append(token)
    
    # Take the first reasonable entity candidate
    entity = entity_candidates[0].lower() if entity_candidates else ""
    
    return entity, detected_relation

def get_relevant_keys_values(entity, relation, DB, max_pairs=10):
    """
    Get relevant key-value pairs for a given entity and relation
    """
    keys = []
    values = []
    target_idx = -1
    
    # Find the person in the database
    person_data = None
    for name, data in DB.items():
        if entity in name.lower():
            person_data = data
            entity = name  # Use the exact name from DB
            break
    
    if not person_data:
        return keys, values, target_idx, None
    
    # Add the target key-value pair if relation exists
    if relation and relation in person_data:
        key_text = f"{entity} {relation}"
        value_text = person_data[relation]
        
        keys.append(multihot(key_text, VOCAB))
        values.append(multihot(value_text, VOCAB))
        target_idx = 0
    
    # Add other relations for the same person
    other_relations = [r for r in person_data.keys() if r != relation]
    random.shuffle(other_relations)
    
    for other_rel in other_relations[:max_pairs-1]:
        other_key_text = f"{entity} {other_rel}"
        other_value_text = person_data[other_rel]
        
        keys.append(multihot(other_key_text, VOCAB))
        values.append(multihot(other_value_text, VOCAB))
    
    # Add some random key-value pairs from other people
    if len(keys) < max_pairs:
        other_names = [name for name in DB.keys() if name != entity]
        random.shuffle(other_names)
        
        for other_name in other_names[:max_pairs - len(keys)]:
            other_person_data = DB[other_name]
            other_relations = list(other_person_data.keys())
            if other_relations:
                random_rel = random.choice(other_relations)
                random_key_text = f"{other_name} {random_rel}"
                random_value_text = other_person_data[random_rel]
                
                keys.append(multihot(random_key_text, VOCAB))
                values.append(multihot(random_value_text, VOCAB))
    
    return keys, values, target_idx, person_data[relation] if relation and relation in person_data else None

def answer_question(question, model, DB, vocab, Y_matrix):
    """
    Answer a natural language question using the trained KVMemNet model
    
    Args:
        question: Natural language question string
        model: Trained KVMemNet model
        DB: Database dictionary
        vocab: Vocabulary object
        Y_matrix: Matrix of all possible values
    
    Returns:
        answer_text: The predicted answer as text
    """
    model.eval()
    
    # Extract entity and relation from question
    entity, relation = extract_entity_and_relation(question)
    print(f"Extracted entity: '{entity}', relation: '{relation}'")
    
    # Get relevant key-value pairs
    keys, values, target_idx, correct_answer = get_relevant_keys_values(entity, relation, DB)
    
    if not keys:
        return "Sorry, I couldn't find information about that person."
    
    # Convert question to bag of words
    question_bow = multihot(question, vocab)
    question_tensor = torch.tensor(question_bow, dtype=torch.float32).to(device)
    
    # Convert keys and values to tensors
    keys_tensor = torch.stack([torch.tensor(k, dtype=torch.float32) for k in keys]).to(device)
    values_tensor = torch.stack([torch.tensor(v, dtype=torch.float32) for v in values]).to(device)
    
    with torch.no_grad():
        # Forward pass through model
        output = model(question_tensor, keys_tensor, values_tensor)
        
        # Embed all values in Y using layer B
        Y_embedded = model.B(Y_matrix)
        
        # Compute similarity scores
        similarity_scores = torch.matmul(Y_embedded, output)
        
        # Get the predicted answer
        predicted_idx = torch.argmax(similarity_scores).item()
        predicted_bow = Y_matrix[predicted_idx].cpu().numpy()
        
        # Convert back to text (this is a simplified approach)
        # In practice, you'd want to store the original text with each bow vector
        predicted_tokens = []
        for i, val in enumerate(predicted_bow):
            if val > 0:
                predicted_tokens.append(vocab.index2word(i))
        
        predicted_text = ' '.join(predicted_tokens)
        
        # Show debug info
        print(f"Predicted index: {predicted_idx}")
        print(f"Correct answer: {correct_answer}")
        print(f"Predicted tokens: {predicted_tokens}")
        
        return predicted_text

# Test the question answering function
print("Testing question answering function...")

# Test with the provided question
test_question = "When was alexander hamilton born?"
answer = answer_question(test_question, real_model, DB, VOCAB, Y_matrix)
print(f"Question: {test_question}")
print(f"Answer: {answer}")
print()

# Test with a few more questions
test_questions = [
    "Where was Alexander Hamilton born?",
    "What party was Alexander Hamilton in?",
    "Who was Alexander Hamilton married to?",
    "What office did Alexander Hamilton hold?"
]

for q in test_questions:
    try:
        answer = answer_question(q, real_model, DB, VOCAB, Y_matrix)
        print(f"Question: {q}")
        print(f"Answer: {answer}")
        print()
    except Exception as e:
        print(f"Error with question '{q}': {e}")
        print()

Testing question answering function...


NameError: name 'Y_matrix' is not defined

# Part F: Reporting (15 points)

Your report should answer the following three questions:

 **Q1:** What pre-processing of the data did you do? What motivated the design decisions and how did it impact training and any processing of natural language questions (Parts A and D)?

 Hint: This should help one understand any code modifications you made in Parts A and the first part of Part D. But you shouldn't use this to document your code (hopefully you commented your code with code comments and text cells above), but to justify your choices as well as to explain what worked and what didn't work.

 **Q2:** Report on your training on the real data (Part D). Show your loss curve and report on the testing accuracy. There are many ways to implement the training loop, particularly with the choice of keys and values. What decisions did you make when developing your training loop? Justify your decisions. How did they impact the training?

 Hint: This assignment doesn't grade you on how well your model learns---your solution will not be perfect. We focus more on how you worked through the process. This part of the report should show how well your solution worked, but also the intuition for why it works, and to document the things you tried that didn't work.

 **Q3:** Describe your technique on how you process natural language questions (Part E). Provide some examples of your technique answering questions correctly and some examples of your technique answering questions incorrectly. Discuss what causes the failure cases.

 Hint: You are not penalized for incorrectly answered questions---your model will not be perfect---we are looking for honest reflection. Preferably, show the example as code blocks running your model with notebook outputs saved.

 We have provided three prompts below. You can create as many text and code cells as necessary.

**Q1: Report on Data Pre-processing**

Your text here.

**Q2: Report on Training and Testing**

Your text here.

**Q3: Report on Model Use**

Your text here.